In [23]:
import numpy as np
import cv2
import os
from glob import glob
import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate

In [11]:
def unet_model(input_shape=(256, 256, 3), num_classes=1):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    # Decoder
    up2 = Concatenate()([UpSampling2D(size=(2, 2))(conv1), conv1])
    conv2 = Conv2D(64, 3, activation='relu', padding='same')(up2)
    conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv2)

    outputs = Conv2D(num_classes, 1, activation='sigmoid')(conv2)  # Output binary mask

    model = Model(inputs=inputs, outputs=outputs)
    return model


In [12]:
data_path = 'C:/crosswalk mask semantic'

In [24]:
images = []
masks = []
for i in tqdm.tqdm(os.listdir(data_path+'/train/')):
    if i[-3:] == 'jpg':
        images.append(cv2.imread(data_path+'/train/'+i))
        masks.append(cv2.imread(data_path+'/train/'+i[:-4]+'_mask.png')[0])

 61%|██████    | 7530/12389 [00:50<00:32, 149.59it/s]


KeyboardInterrupt: 

In [4]:
def load_and_preprocess_data(images, masks, target_size=(640, 640)):
    num_samples = len(images)
    X = np.zeros((num_samples, *target_size, 3), dtype=np.float32)  # Input images
    Y = np.zeros((num_samples, *target_size, 1), dtype=np.float32)  # Binary masks

    for idx in range(num_samples):
        # Load and preprocess the input image
        img = images[idx]  # Load the image (you need to implement this part)
        img = cv2.resize(img, target_size)  # Resize the image to the target size
        img = img / 255.0  # Normalize pixel values to 0-1
        X[idx] = img

        # Load and preprocess the binary mask
        mask = masks[idx]  # Load the binary mask (you need to implement this part)
        mask = cv2.resize(mask, target_size)  # Resize the mask to the target size
        mask = np.expand_dims(mask, axis=-1)  # Add channel dimension
        mask = mask.astype(np.float32)  # Ensure the data type is float32
        Y[idx] = mask

    return X, Y

# 예시 데이터 로딩 및 전처리
# images, masks = load_data_from_disk()  # 이미지와 바이너리 마스크 데이터를 디스크에서 로드하는 함수 (실제 데이터에 맞게 구현 필요)
# X_train, Y_train = load_and_preprocess_data(images, masks)


In [ ]:

model = unet_model()
model.compile(optimizer=Adam(lr=1e-4), loss=binary_crossentropy, metrics=['accuracy'])

# 데이터 로딩 및 전처리
X_train, Y_train = load_and_preprocess_data()

# 모델 학습
model.fit(X_train, Y_train, batch_size=32, epochs=50, validation_split=0.1)
